In [1]:
import pandas as pd
df = pd.read_csv('../datasets/kingcounty/raw/kc_house_data.csv');
df.sample(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
13681,7503800210,20150326T000000,295000.0,4,2.5,1677,7209,2.0,0,0,...,7,1677,0,2011,0,98023,47.2957,-122.357,2236,7209
3425,5363200266,20140623T000000,420000.0,3,2.0,1200,5029,1.0,0,0,...,6,880,320,1937,0,98115,47.6936,-122.294,1510,5854
12703,9164100040,20141026T000000,390000.0,2,1.0,860,5160,1.0,0,0,...,7,860,0,1909,0,98117,47.6823,-122.388,1090,5356


In [2]:
columns = ('bathrooms', 'bedrooms', 'condition', 'floors',  'grade', 'id', 'lat', 'long',  'price', 'sqft_above', 'sqft_basement', 'sqft_living', 'sqft_living15', 'sqft_lot',
 'sqft_lot15',
 'view',
 'waterfront',
 'yr_built',
 'yr_renovated',
 'zipcode')

In [3]:
df.loc[:, columns].drop(columns=('price'))

,bathrooms,bedrooms,condition,floors,grade,id,lat,long,sqft_above,sqft_basement,sqft_living,sqft_living15,sqft_lot,sqft_lot15,view,waterfront,yr_built,yr_renovated,zipcode
0,1.00,3,3,1.0,7,7129300520,47.5112,-122.257,1180,0,1180,1340,5650,5650,0,0,1955,0,98178
1,2.25,3,3,2.0,7,6414100192,47.7210,-122.319,2170,400,2570,1690,7242,7639,0,0,1951,1991,98125
2,1.00,2,3,1.0,6,5631500400,47.7379,-122.233,770,0,770,2720,10000,8062,0,0,1933,0,98028
3,3.00,4,5,1.0,7,2487200875,47.5208,-122.393,1050,910,1960,1360,5000,5000,0,0,1965,0,98136
4,2.00,3,3,1.0,8,1954400510,47.6168,-122.045,1680,0,1680,1800,8080,7503,0,0,1987,0,98074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,2.50,3,3,3.0,8,263000018,47.6993,-122.346,1530,0,1530,1530,1131,1509,0,0,2009,0,98103
21609,2.50,4,3,2.0,8,6600060120,47.5107,-122.362,2310,0,2310,1830,5813,7200,0,0,2014,0,98146
21610,0.75,2,3,2.0,7,1523300141,47.5944,-122.299,1020,0,1020,1020,1350,2007,0,0,2009,0,98144
21611,2.50,3,3,2.0,8,291310100,47.5345,-122.069,1600,0,1600,1410,2388,1287,0,0,2004,0,98027


In [4]:
import os
import lib
import argparse
import numpy as np
from pathlib import Path
from typing import Union, Any
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import check_random_state


In [5]:
pd.DataFrame([], columns=["A", "B"])

,A,B


In [6]:
from syntheticml.models.smote.smote import MySMOTENC, MySMOTE
import numpy as np

cat_columns=("condition", "floors", "grade", "view", "waterfront", "zipcode", "bathrooms", "bedrooms",)
exclude_columns=('date', 'id')
target_column = "price"

columns = list(set(df.columns) - set(exclude_columns))
num_columns = list(set(columns) - set(cat_columns) - set([target_column]) )

n_sample = df.shape[0]
eval_type = "synthetic"
k_neighbours = 5

frac_samples = df.shape[0] / n_sample
frac_lam_del = 0.0
save = True
seed = 42


lam1 = 0.0 + frac_lam_del / 2
lam2 = 1.0 - frac_lam_del / 2
is_regression = True




if is_regression:
    X_train = df.loc[:, num_columns + [target_column]]
    y_train = np.where( df.loc[:, target_column] > np.median(df.loc[:, target_column]), 1, 0)
else:
    X_train = df.drop(columns=[target_column])
    y_train = df.loc[:, target_column]

n_num_features = len(num_columns)
n_cat_features = len(cat_columns)
cat_features = list(range(n_num_features, n_num_features+n_cat_features))
display(X_train.head(2))

scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train).astype(object)
display(pd.DataFrame(X_train_scaled[0:2,:], columns=X_train.columns))

X_train_scaled_cat = np.concatenate( [X_train_scaled, df.loc[:, cat_columns].to_numpy()] , axis=1, dtype=object)
sorted_columns = num_columns + [target_column] + list(cat_columns)
display(pd.DataFrame(X_train_scaled_cat[0:2,:], columns=sorted_columns))

y_cats, y_counts = np.unique(y_train, return_counts=True)
strat = dict(zip(y_cats, (y_counts*(1+frac_samples)).astype(int) ))
display(strat)

if len(cat_columns) > 0:
    sm = MySMOTENC(
        lam1=lam1,
        lam2=lam2,
        random_state=seed,
        k_neighbors=k_neighbours,
        categorical_features=cat_features,
        sampling_strategy=strat
    )
else:
    sm = MySMOTE(
            lam1=lam1,
            lam2=lam2,
            random_state=seed,
            k_neighbors=k_neighbours,
    )    
X_res, y_res = sm.fit_resample(X_train_scaled_cat, y_train)
outsample = X_res[X_train.shape[0]:]
num_outsample = scaler.inverse_transform(outsample[:, :n_num_features+1])
cat_outsample = outsample[:, n_num_features+1:]
complete_outsample = np.concatenate([num_outsample, cat_outsample], axis=1)
new_sample = pd.DataFrame(complete_outsample, columns=sorted_columns)

,sqft_above,yr_built,long,lat,sqft_living,sqft_living15,sqft_lot15,sqft_lot,sqft_basement,yr_renovated,price
0,1180,1955,-122.257,47.5112,1180,1340,5650,5650,0,0,221900.0
1,2170,1951,-122.319,47.7210,2570,1690,7639,7242,400,1991,538000.0


,sqft_above,yr_built,long,lat,sqft_living,sqft_living15,sqft_lot15,sqft_lot,sqft_basement,yr_renovated,price
0,0.097588,0.478261,0.217608,0.571498,0.06717,0.161934,0.005742,0.003108,0.0,0.0,0.019266
1,0.20614,0.443478,0.166113,0.908959,0.172075,0.222165,0.008027,0.004072,0.082988,0.988089,0.060721


,sqft_above,yr_built,long,lat,sqft_living,sqft_living15,sqft_lot15,sqft_lot,sqft_basement,yr_renovated,price,condition,floors,grade,view,waterfront,zipcode,bathrooms,bedrooms
0,0.097588,0.478261,0.217608,0.571498,0.06717,0.161934,0.005742,0.003108,0.0,0.0,0.019266,3.0,1.0,7.0,0.0,0.0,98178.0,1.0,3.0
1,0.20614,0.443478,0.166113,0.908959,0.172075,0.222165,0.008027,0.004072,0.082988,0.988089,0.060721,3.0,2.0,7.0,0.0,0.0,98125.0,2.25,3.0


{0: 21728, 1: 21498}

/home/gvillarroel/.pyenv/versions/3.9.15/envs/syn/lib/python3.9/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (21728) in class 0 will be larger than the number of samples in the majority class (class #0 -> 10864)
  warnings.warn(
/home/gvillarroel/.pyenv/versions/3.9.15/envs/syn/lib/python3.9/site-packages/imblearn/utils/_validation.py:299: UserWarning: After over-sampling, the number of samples (21498) in class 1 will be larger than the number of samples in the majority class (class #0 -> 10864)
  warnings.warn(


In [12]:
import rdt
transformer = rdt.transformers.FrequencyEncoder(add_noise=True)

In [15]:
mu, sigma = 0, 0.1 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)

In [21]:
import plotly.graph_objects as go
fig = go.Figure( data=[
        go.Histogram(x=s, name="Real", marker_color="blue")
        ], layout=dict(title="S", barmode='overlay'))

fig

In [7]:
pd.read_parquet("/home/gvillarroel/dev/synthetic-data-for-text/datasets/kingcounty/synth/data/smote-enc_noise_21613.parquet")

,id,sqft_living,sqft_lot,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,...,price,date,bedrooms,bathrooms,floors,waterfront,view,condition,grade,zipcode
0,0,2530.015264,6229.741044,2530.015264,0.000000,2002.000763,0.0,47.480549,-122.183997,2564.996184,...,445000.0,20150317T000000,4,2.50,2.0,0,0,3,7,98056.0
1,1,1803.573598,8326.173585,1348.312872,455.260727,1981.915644,0.0,47.446186,-122.123000,1910.000000,...,315000.0,20150422T000000,3,1.75,1.0,0,0,3,8,98058.0
2,2,1184.726335,5658.323601,1184.726335,0.000000,1954.973743,0.0,47.510835,-122.256974,1340.525148,...,241000.0,20140716T000000,3,1.00,1.0,0,0,3,6,98178.0
3,3,1228.715181,9460.536129,1228.715181,0.000000,1985.712923,0.0,47.358767,-122.285644,1646.633626,...,231000.0,20140528T000000,3,2.50,1.0,0,0,3,7,98032.0
4,4,1440.000000,4464.844129,1440.000000,0.000000,2003.000000,0.0,47.681922,-122.034408,1596.741377,...,440500.0,20141024T000000,3,2.50,2.0,0,0,3,7,98053.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21607,21607,3150.000000,7368.616945,3150.000000,0.000000,2001.000000,0.0,47.596458,-122.017380,3150.000000,...,700000.0,20140902T000000,4,3.00,2.0,0,0,3,9,98075.0
21608,21608,3538.640364,5104.803481,2783.401208,755.239156,2013.040301,0.0,47.554152,-122.358798,2874.131932,...,705000.0,20150429T000000,5,3.50,2.0,0,0,3,8,98106.0
21609,21609,1936.581920,10715.363231,1244.149097,692.432823,1952.756718,0.0,47.594822,-122.199018,2059.462587,...,650000.0,20150408T000000,3,1.75,1.0,0,0,3,7,98004.0
21610,21610,2471.364425,16740.128170,1907.981426,563.382999,1936.355138,0.0,47.680557,-122.403503,2285.569957,...,715000.0,20140528T000000,4,2.00,1.5,0,4,4,9,98117.0


In [8]:
new_sample.head()

,sqft_above,yr_built,long,lat,sqft_living,sqft_living15,sqft_lot15,sqft_lot,sqft_basement,yr_renovated,price,condition,floors,grade,view,waterfront,zipcode,bathrooms,bedrooms
0,2960.0,2005.0,-122.04,47.359621,2960.0,2607.695181,4930.536526,5739.70002,0.0,0.0,440000.0,3.0,2.0,9.0,0.0,0.0,98038.0,2.5,4.0
1,960.0,1922.0,-122.352,47.4957,1240.0,1820.0,7460.0,12092.0,280.0,1984.0,224000.0,3.0,1.0,6.0,0.0,0.0,98168.0,1.0,2.0
2,1522.995734,1952.211658,-122.338095,47.761766,1522.995734,1424.60108,8764.206991,8658.456573,0.0,0.0,400000.0,4.0,1.5,7.0,0.0,0.0,98133.0,1.0,4.0
3,741.180576,1944.136571,-122.306863,47.696607,741.180576,1608.819424,7100.156629,7100.156629,0.0,0.0,400000.0,3.0,1.0,6.0,0.0,0.0,98115.0,1.0,2.0
4,1231.50963,1965.528301,-122.178245,47.455583,1231.50963,1407.924345,7659.849106,7698.943398,0.0,0.0,255000.0,4.0,1.0,7.0,0.0,0.0,98058.0,1.5,3.0


In [9]:
from syntheticml.models.smote.sdv import SDVSMOTE
model = SDVSMOTE.load(filepath="/home/gvillarroel/dev/synthetic-data-for-text/datasets/kingcounty/synth/checkpoint/smote-enc_noise.ckp")
s = model.sample

In [12]:
import pandas as pd
pd.read_parquet("/home/gvillarroel/dev/synthetic-data-for-text/datasets/kingcounty/synth/data/smote-enc_21613_frac_lam_del=0.1.parquet")

,id,sqft_living,sqft_lot,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,...,price,date,bedrooms,bathrooms,floors,waterfront,view,condition,grade,zipcode
0,0,2530.015264,6229.741044,2530.015264,0.000000,2002.000763,0.0,47.480549,-122.183997,2564.996184,...,445000.0,20150317T000000,4,2.50,2.0,0,0,3,7,98056.0
1,1,1803.573598,8326.173585,1348.312872,455.260727,1981.915644,0.0,47.446186,-122.123000,1910.000000,...,315000.0,20150422T000000,3,1.75,1.0,0,0,3,8,98058.0
2,2,1184.726335,5658.323601,1184.726335,0.000000,1954.973743,0.0,47.510835,-122.256974,1340.525148,...,241000.0,20140716T000000,3,1.00,1.0,0,0,3,6,98178.0
3,3,1228.715181,9460.536129,1228.715181,0.000000,1985.712923,0.0,47.358767,-122.285644,1646.633626,...,231000.0,20140528T000000,3,2.50,1.0,0,0,3,7,98032.0
4,4,1440.000000,4464.844129,1440.000000,0.000000,2003.000000,0.0,47.681922,-122.034408,1596.741377,...,440500.0,20141024T000000,3,2.50,2.0,0,0,3,7,98053.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21607,21607,3150.000000,7368.616945,3150.000000,0.000000,2001.000000,0.0,47.596458,-122.017380,3150.000000,...,700000.0,20140902T000000,4,3.00,2.0,0,0,3,9,98075.0
21608,21608,3538.640364,5104.803481,2783.401208,755.239156,2013.040301,0.0,47.554152,-122.358798,2874.131932,...,705000.0,20150429T000000,5,3.50,2.0,0,0,3,8,98106.0
21609,21609,1936.581920,10715.363231,1244.149097,692.432823,1952.756718,0.0,47.594822,-122.199018,2059.462587,...,650000.0,20150408T000000,3,1.75,1.0,0,0,3,7,98004.0
21610,21610,2471.364425,16740.128170,1907.981426,563.382999,1936.355138,0.0,47.680557,-122.403503,2285.569957,...,715000.0,20140528T000000,4,2.00,1.5,0,4,4,9,98117.0


In [10]:
display(pd.DataFrame(X_res, columns=sorted_columns))

,sqft_above,yr_built,long,lat,sqft_living,sqft_living15,sqft_lot15,sqft_lot,sqft_basement,yr_renovated,price,condition,floors,grade,view,waterfront,zipcode,bathrooms,bedrooms
0,0.097588,0.478261,0.217608,0.571498,0.06717,0.161934,0.005742,0.003108,0.0,0.0,0.019266,3.0,1.0,7.0,0.0,0.0,98178.0,1.0,3.0
1,0.20614,0.443478,0.166113,0.908959,0.172075,0.222165,0.008027,0.004072,0.082988,0.988089,0.060721,3.0,2.0,7.0,0.0,0.0,98125.0,2.25,3.0
2,0.052632,0.286957,0.237542,0.936143,0.036226,0.399415,0.008513,0.005743,0.0,0.0,0.01377,3.0,1.0,6.0,0.0,0.0,98028.0,1.0,2.0
3,0.083333,0.565217,0.104651,0.586939,0.126038,0.165376,0.004996,0.002714,0.188797,0.0,0.069377,5.0,1.0,7.0,0.0,0.0,98136.0,3.0,4.0
4,0.152412,0.756522,0.393688,0.741354,0.104906,0.241094,0.007871,0.004579,0.0,0.0,0.057049,3.0,1.0,8.0,0.0,0.0,98074.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43221,0.294602,0.817687,0.375232,0.656796,0.202775,0.449525,0.011043,0.00529,0.0,0.0,0.096131,3.0,2.0,10.0,0.0,0.0,98027.0,2.5,4.0
43222,0.254603,0.798008,0.332036,0.638767,0.175244,0.428292,0.009983,0.005419,0.0,0.0,0.096393,3.0,2.0,9.0,0.0,0.0,98006.0,2.5,3.0
43223,0.294626,0.979152,0.336185,0.883823,0.202792,0.448919,0.00479,0.002525,0.0,0.0,0.091934,3.0,2.0,8.0,0.0,0.0,98052.0,2.5,4.0
43224,0.239961,0.946117,0.345337,0.878166,0.165165,0.378187,0.004129,0.002114,0.0,0.0,0.079069,3.0,2.0,8.0,0.0,0.0,98052.0,2.5,3.0


In [11]:
display(X_train_scaled_cat.shape[0])
display(pd.DataFrame(X_train_scaled_cat[-1:,:], columns=sorted_columns))
display(pd.DataFrame(X_res[X_train_scaled_cat.shape[0]:,:], columns=sorted_columns))


21613

,sqft_above,yr_built,long,lat,sqft_living,sqft_living15,sqft_lot15,sqft_lot,sqft_basement,yr_renovated,price,condition,floors,grade,view,waterfront,zipcode,bathrooms,bedrooms
0,0.080044,0.93913,0.182724,0.704842,0.055094,0.106866,0.000811,0.000337,0.0,0.0,0.032787,3.0,2.0,7.0,0.0,0.0,98144.0,0.75,2.0


,sqft_above,yr_built,long,lat,sqft_living,sqft_living15,sqft_lot15,sqft_lot,sqft_basement,yr_renovated,price,condition,floors,grade,view,waterfront,zipcode,bathrooms,bedrooms
0,0.292763,0.913043,0.397841,0.327684,0.201509,0.380089,0.004916,0.003162,0.0,0.0,0.047869,3.0,2.0,9.0,0.0,0.0,98038.0,2.5,4.0
1,0.073465,0.191304,0.138704,0.546566,0.071698,0.244536,0.007822,0.00701,0.058091,0.984615,0.019541,3.0,1.0,6.0,0.0,0.0,98168.0,1.0,2.0
2,0.135197,0.454014,0.150253,0.974532,0.093056,0.176493,0.00932,0.00493,0.0,0.0,0.042623,4.0,1.5,7.0,0.0,0.0,98133.0,1.0,4.0
3,0.049472,0.383796,0.176193,0.869723,0.034051,0.208195,0.007408,0.003986,0.0,0.0,0.042623,3.0,1.0,6.0,0.0,0.0,98115.0,1.0,2.0
4,0.103236,0.569811,0.283019,0.482038,0.071057,0.173623,0.008051,0.004349,0.0,0.0,0.023607,4.0,1.0,7.0,0.0,0.0,98058.0,1.5,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,0.294602,0.817687,0.375232,0.656796,0.202775,0.449525,0.011043,0.00529,0.0,0.0,0.096131,3.0,2.0,10.0,0.0,0.0,98027.0,2.5,4.0
21609,0.254603,0.798008,0.332036,0.638767,0.175244,0.428292,0.009983,0.005419,0.0,0.0,0.096393,3.0,2.0,9.0,0.0,0.0,98006.0,2.5,3.0
21610,0.294626,0.979152,0.336185,0.883823,0.202792,0.448919,0.00479,0.002525,0.0,0.0,0.091934,3.0,2.0,8.0,0.0,0.0,98052.0,2.5,4.0
21611,0.239961,0.946117,0.345337,0.878166,0.165165,0.378187,0.004129,0.002114,0.0,0.0,0.079069,3.0,2.0,8.0,0.0,0.0,98052.0,2.5,3.0
